In [ ]:
import pandas as pd
import numpy as np
import random

import openai
import os
from tqdm import tqdm
import time

import sys

from sklearn.metrics import accuracy_score, classification_report

sys.path.insert(0, '..')

In [ ]:
# TURBO COST
TURBO_COST = 0.00200 / 1000

In [ ]:
# Initialize the API client
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
finance = pd.read_csv('../data/seed/youtube/seed_videos.csv')
young = pd.read_csv('../data/seed/youtube/young_videos.csv')
old = pd.read_csv('../data/seed/youtube/old_videos.csv')

In [ ]:
df = pd.concat([finance, young, old])

In [ ]:
df['label'] = df['label'].apply(lambda x: 'Unrelated' if x == 'old' else 'Unrelated' if x == 'young' else 'Unrelated' if x == 'None' else x)

In [ ]:
def create_messages(snippet):
    messages = [
        {"role": "system", "content" : "You are a classifier that determines if a YouTube video snippet falls under a label. A snippet is a concatenation of the video title, summarized transcript, and video tags. The labels and additional instructions will be included in the first user message."},
        {"role": "user", "content" : """Labels:

Traditional: Videos that recommend or educate about stocks, bonds, real estate, commodities, retirement accounts, or other traditional investments or keywords related to them.
Blockchain: Videos that recommend or educate about cryptocurrency (BTC, ETH, etc.), NFTs, or other Web3 investments or keywords related to them.
Mixed: Videos that recommend or educate about both blockchain and traditional investments or keywords related to both.
Unrelated: Videos that do not recommend or educate about either blockchain or traditional investments or keywords related to them.

Instructions:
- The classifier should consider the context and meaning of the keywords used to determine whether the snippet is related to traditional or blockchain investments.
- If talks about making money from jobs, side hustles, or other alternative assets (cars, watches, artificial intelligence, trading cards, art, etc), they are Unrelated.
- A video that is only downplaying an investment or discussing it negatively should be classified as Unrelated.
- Please return predictions in the format" {Label} : {20 word or shorter rationale}"""},
        {"role": "assistant", "content": """Understood. I will classify YouTube video snippets based on the provided labels and instructions. Here's how I will format the predictions:

{Label} : {20-word or shorter rationale}

Please provide me with the YouTube video snippet you would like me to classify."""},
    ]
    
    snippet_message = {"role" : "user", "content" : snippet.replace("\n", " ").replace("  ", " ")}
    
    messages.append(snippet_message)
    
    return messages

In [ ]:
def classify(messages, temp = 0.25):
    chatCompletion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=temp
    )
    return chatCompletion

In [ ]:
df['messages'] = df['snippet'].apply(create_messages)

In [ ]:
completions = []

pbar = tqdm(df.iterrows())  

for idx, entry in pbar:
    pbar.set_description("Processing %s" % entry['title'])
    
    body = classify(entry['messages'])
    
    ## Add error catching
    completion = dict(body['choices'][0]['message'])
    completion['prediction']  = completion['content'].split(':')[0].strip()
    completion['reason'] = completion['content'].split(':')[1].strip()
    completion['message'] = entry['messages']
    completion['title'] = entry['title']
    completion['transcript'] = entry['cleaned_transcript']
    completion['snippet'] = entry['snippet']
    completion['link'] = entry['link']

    ## Grab meta data
    completion.update({key: body[key] for key in ['created', 'id', 'model', 'object']})
    
    ## Grab token usage
    completion.update(dict(body['usage']))

    completions.append(completion)
    
    pd.DataFrame(completions).to_csv('seed_predictions.csv', index_label = False)
    
    
    time.sleep(5)

In [ ]:
predictions = pd.read_csv('seed_predictions.csv')

In [ ]:
predictions['prediction'].value_counts()

In [ ]:
preds = list(predictions['prediction'])
actual = list(df['label'])
preds = ['Blockchain' if pred == 'Label' else pred for pred in preds]
sum(predictions['total_tokens']) * TURBO_COST

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
cf_matrix = confusion_matrix(actual, preds)

In [ ]:
cf_matrix

In [ ]:
import matplotlib.pyplot as plt     

ax= plt.subplot()

sns.heatmap(cf_matrix, annot=True)

# labels, title and ticks
ax.set_xlabel('Predicted Labels');ax.set_ylabel('True Labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Blockchain', 'Mixed', 'Traditional', 'Unrelated']); ax.yaxis.set_ticklabels(['Traditional', 'Blockchain', 'Mixed', 'Unrelated']);

In [ ]:
def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in
    group_names:   List of strings that represent the labels row by row to be shown in each square.
    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'
    count:         If True, show the raw number in the confusion matrix. Default is True.
    normalize:     If True, show the proportions for each category. Default is True.
    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.
    xyticks:       If True, show x and y ticks. Default is True.
    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.
    sum_stats:     If True, display summary statistics below the figure. Default is True.
    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.
    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
                   
    title:         Title for the heatmap. Default is None.
    '''


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf)) * 100

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}%".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    output = sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

    return output

In [ ]:
labels = ['Blockchain', 'Mixed', 'Traditional', 'Unrelated']
cf_matrix = confusion_matrix(actual, preds)

make_confusion_matrix(cf_matrix, categories = labels, cmap = "rocket").get_figure().savefig("seed_training_results.png", dpi = 300, bbox_inches ='tight')

In [ ]:
print(accuracy_score(actual, preds))
print(classification_report(actual, preds))

In [ ]:
pred = 'Label :Blockchain bruh'

In [ ]:
pred = pred.lower().strip('label').strip('label:').strip().strip(":").strip().split(" ")[0]

In [ ]:
pred = pred[0].upper() + pred[1:]

In [ ]:
pred

In [ ]:

df = pd.read_csv("../data/audit/youtube/processed/snippets/downloaded_sidebar_mixed_old_with_snippets.csv")

In [ ]:
create_messages(df.iloc[545]['snippet'])

In [ ]:
classify(create_messages(df.iloc[545]['snippet']))